In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

url_target = 'https://standings.stalruth.dev/2025/regional-sacramento/masters'
page = urlopen(url_target)
soup = BeautifulSoup(page)

In [2]:
standing_table = soup.findAll('table')[1]

In [3]:
standing_rows = standing_table.findAll('tr')

In [4]:
standing_rows[0].find('td', attrs={'class': 'name-cell'})

<td class="name-cell">
<a href="/2025/regional-sacramento/masters/446">
              Carson Confer [US]
            </a>
</td>

In [10]:
mons_usage_wr_dict = {}


In [6]:
drop_form_names = ['Sinistcha', 'Tatsugiri', 'Maushold']

In [12]:
for r in standing_rows:
    mons_ = r.find_all('span', attrs={'class': 'pokemon-icon'})
    record_ = r.find('td', attrs={'class': 'record-resistance'}).find('p', attrs={'class': 'record'}).get_text()
    for m in mons_:
        mon_name = m.get('title')
        # Drop the form names
        for dpn in drop_form_names:
            if dpn in mon_name:
                mon_name = dpn
        wins = int(record_.split('-')[0])
        losses = int(record_.split('-')[1].split('*')[0])
        if not (mon_name in mons_usage_wr_dict.keys()):
            mons_usage_wr_dict[mon_name] = {'win': wins,
                                    'loss': losses,
                                    'day1': 1,
                                    'day2': 1 if (wins > 5) else 0
                                    }
        else:
            # print(mons_usage_wr_dict[mon_name], record_)
            mons_usage_wr_dict_temp = {'win': mons_usage_wr_dict[mon_name]['win'] + wins,
                                        'loss': mons_usage_wr_dict[mon_name]['loss'] + losses,
                                        'day1': mons_usage_wr_dict[mon_name]['day1'] + 1,
                                        'day2': mons_usage_wr_dict[mon_name]['day2'] + (1 if (wins > 5) else 0)
                                        }
            mons_usage_wr_dict[mon_name] = mons_usage_wr_dict_temp


In [13]:
mons_usage_wr_dict

{'Sneasler': {'win': 1066, 'loss': 980, 'day1': 247, 'day2': 44},
 'Electabuzz': {'win': 343, 'loss': 351, 'day1': 87, 'day2': 10},
 'Ursaluna-Bloodmoon': {'win': 449, 'loss': 400, 'day1': 102, 'day2': 20},
 'Incineroar': {'win': 775, 'loss': 702, 'day1': 178, 'day2': 32},
 'Talonflame': {'win': 174, 'loss': 141, 'day1': 39, 'day2': 8},
 'Indeedee': {'win': 226, 'loss': 200, 'day1': 50, 'day2': 9},
 'Garchomp': {'win': 227, 'loss': 215, 'day1': 55, 'day2': 8},
 'Jumpluff': {'win': 108, 'loss': 111, 'day1': 27, 'day2': 4},
 'Torkoal': {'win': 203, 'loss': 233, 'day1': 55, 'day2': 7},
 'Dragonite': {'win': 746, 'loss': 638, 'day1': 169, 'day2': 29},
 'Kingambit': {'win': 459, 'loss': 495, 'day1': 121, 'day2': 14},
 'Typhlosion-Hisui': {'win': 107, 'loss': 98, 'day1': 25, 'day2': 5},
 'Archaludon': {'win': 707, 'loss': 681, 'day1': 171, 'day2': 27},
 'Amoonguss': {'win': 485, 'loss': 421, 'day1': 108, 'day2': 24},
 'Corviknight': {'win': 80, 'loss': 79, 'day1': 20, 'day2': 3},
 'Hydreigon

In [22]:
mons_usage_winrate_df = pd.DataFrame(mons_usage_wr_dict).T

In [23]:
mons_usage_winrate_df

,win,loss,day1,day2
Sneasler,1066,980,247,44
Electabuzz,343,351,87,10
Ursaluna-Bloodmoon,449,400,102,20
Incineroar,775,702,178,32
Talonflame,174,141,39,8
...,...,...,...,...
Luxray,0,3,1,0
Bellibolt,0,3,1,0
Weezing,0,3,1,0
Feraligatr,0,2,1,0


In [24]:
mons_usage_winrate_df['win_rate'] = mons_usage_winrate_df['win']/(mons_usage_winrate_df['win'] + mons_usage_winrate_df['loss'])

In [25]:
mons_usage_winrate_df['conversion_rate(day2:day1)'] = (mons_usage_winrate_df['day2']/86) / (mons_usage_winrate_df['day1']/590)

In [26]:
mons_usage_winrate_df['day1']

Sneasler              247
Electabuzz             87
Ursaluna-Bloodmoon    102
Incineroar            178
Talonflame             39
                     ... 
Luxray                  1
Bellibolt               1
Weezing                 1
Feraligatr              1
Glaceon                 1
Name: day1, Length: 168, dtype: int64

In [27]:
mons_usage_winrate_df = mons_usage_winrate_df.round(4)

In [28]:
mons_usage_winrate_df

,win,loss,day1,day2,win_rate,conversion_rate(day2:day1)
Sneasler,1066,980,247,44,0.5210,1.2221
Electabuzz,343,351,87,10,0.4942,0.7886
Ursaluna-Bloodmoon,449,400,102,20,0.5289,1.3452
Incineroar,775,702,178,32,0.5247,1.2333
Talonflame,174,141,39,8,0.5524,1.4073
...,...,...,...,...,...,...
Luxray,0,3,1,0,0.0000,0.0000
Bellibolt,0,3,1,0,0.0000,0.0000
Weezing,0,3,1,0,0.0000,0.0000
Feraligatr,0,2,1,0,0.0000,0.0000


In [29]:
mons_usage_winrate_df.to_csv('output_txt/sac24_mons_summary.csv')

In [16]:
sneaslers_players = []
for r in standing_rows:
    if (r.find('span', title='Sneasler') != None):
        name_cell = r.find('td', attrs={'class': 'name-cell'})
        name_ = name_cell.get_text()
        link_ = name_cell.find('a').get('href')
        record_ = r.find('td', attrs={'class': 'record-resistance'}).find('p', attrs={'class': 'record'}).get_text()
        player_dict = {'player_name': name_,
                        'schedule_link_suffix': link_,
                        'record': record_,
                        'win': int(record_.split('-')[0]),
                        'loss': int(record_.split('-')[1])}
        sneaslers_players.append(player_dict)
        # sneaslers_players.append(r)

In [17]:
len(sneaslers_players)

193

In [18]:
positive_records = []
for sneasler_player in sneaslers_players:
    if sneasler_player['win'] >= 5:
        positive_records.append(sneasler_player)

In [19]:
positive_records

[{'player_name': '\n\n              Mattie Morgan [IE]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/341',
  'record': '13-3',
  'win': 13,
  'loss': 3},
 {'player_name': '\n\n              Nicolò Berardo [IT]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/153',
  'record': '12-3',
  'win': 12,
  'loss': 3},
 {'player_name': '\n\n              Ibrahim Maarouf [DE]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/328',
  'record': '11-3',
  'win': 11,
  'loss': 3},
 {'player_name': '\n\n              Nico Davide Cognetta [IT]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/161',
  'record': '11-3',
  'win': 11,
  'loss': 3},
 {'player_name': '\n\n              Joan Garcia [ES]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/450',
  'record': '11-2',
  'win': 11,
  'loss': 2},
 {'player_name': '\n\n              Michael Kelsch [DE]\n          

In [35]:
df_count_sorted = df_count.sort_values(by='count', ascending=False).reset_index(drop=True)

In [38]:
pd.set_option('display.max_rows', 100)

In [39]:
df_count_sorted

,mon,count
0,Rillaboom,104
1,Kingambit,95
2,Gholdengo,89
3,Incineroar,88
4,Primarina,88
5,Sneasler,87
6,Amoonguss,62
7,Dragapult,54
8,Electabuzz,46
9,Garchomp,46


In [37]:
df_count_sorted[df_count_sorted['mon'] == 'Sneasler']

,mon,count
5,Sneasler,87
